In [1]:
# INFERENCE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor,ToPILImage,Normalize
from utils.dataset import LoadCoCoDataset
from utils.model import construct_style_loss_model,construct_decoder_from_encoder,AdaIN,SANet
from utils.losses import content_gatyes,style_gatyes,style_mmd_polynomial,adaIN
from utils.utility import video_to_frame_generator,video_to_frames,normalize,normalize_cw
import cv2
import os
from copy import deepcopy
import torch.utils.data as data
from torchvision.models import vgg19,VGG19_Weights
from PIL import Image,ImageOps
import torch.optim as optim
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
# INFERENCE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On device: ", device)

On device:  cuda


In [3]:
# INFERENCE

IMAGE_SIZE = (256,256)

In [4]:
# INFERENCE

STYLE_LAYERS = [9,12,14]
STYLE_LAYERS_WEIGHTS = [1.0,1.0,1.0]

In [5]:
# INFERENCE

# load standard vgg19 model
vgg = vgg19(VGG19_Weights.DEFAULT)

# remove classification head
vgg = vgg.features

# prepend a normalization layer
vgg = nn.Sequential(Normalize(mean = (0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)), *vgg)

style_loss_model = construct_style_loss_model(vgg,[],STYLE_LAYERS)
style_loss_model = style_loss_model.eval()
style_loss_model.requires_grad_(False)

# as encoders for our feedforward model we will just use the already existing vgg layers up to some point
vgg_encoder_r_4_1 = vgg[:10]
vgg_encoder_r_5_1 = vgg[10:13]

example_input = torch.rand(16,3,*IMAGE_SIZE)
# get the size of the respective encoder outputs
size_r_4_1 = vgg_encoder_r_4_1(example_input).size()[1:]
size_r_5_1 = vgg_encoder_r_5_1(vgg_encoder_r_4_1(example_input)).size()[1:]

# Our Style Model will use the feature output of vgg_encoder_r_5_1 and vgg_encoder_r_4_1. But the vgg_encoder_r_5_1 will be upsampled to the size of vgg_encoder_r_4_1.
# After that a 3x3 convolution is applied and that is fed into our decoder. To make our decoder match that last 3x3 convolution size, 
# we simply add it to the vgg_encoder_r_4_1 when constructing the decoder (NOT IN THE ACTUAL MODEL DIRECTLY AFTER THE ENCODER).
# This simulated encoder is then used to construct the decoder. 
simulated_encoder = nn.Sequential(*vgg[0:10],nn.Conv2d(size_r_4_1[0],size_r_4_1[0],kernel_size=3,stride=3))
# based on that we build a decoder that reverses our encoder and matches the shapes through interpolation
decoder = construct_decoder_from_encoder(simulated_encoder,3,*IMAGE_SIZE)

C:\Users\Leon\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


range(0, 15)


In [6]:
# INFERENCE

class Encoder(nn.Module):
    """
    This encoder returns two feature maps from an encoder. The encoders given as arguments have to be sliced in such a way that the output from encoder 1 can be fed into encoder 2.
    """
    
    def __init__(self, vgg_encoder_r_4_1, vgg_encoder_r_5_1):
        super().__init__()
        self.vgg_encoder_r_4_1 = vgg_encoder_r_4_1
        self.vgg_encoder_r_5_1 = vgg_encoder_r_5_1

    def forward(self, img):
        f_r_4_1 = self.vgg_encoder_r_4_1(img)
        f_r_5_1 = self.vgg_encoder_r_5_1(f_r_4_1)

        return f_r_4_1,f_r_5_1

In [7]:
# INFERENCE

class StyleModel(nn.Module):
    """
    Implementing the full Style Network from the SANet paper. Note that I named the variables after the variables in the model sketch in the paper
    """

    def __init__(self, encoder, size_r_4_1, size_r_5_1,decoder) -> None:
        super().__init__()
        self.encoder = encoder
        self.size_r_4_1 = size_r_4_1

        # initialize SANet with the channel size
        self.SANet_r_4_1 = SANet(size_r_4_1[0])
        self.SANet_r_5_1 = SANet(size_r_5_1[0])
        self.decoder = decoder

        self.conv1 = nn.Conv2d(size_r_4_1[0],size_r_4_1[0],kernel_size=1,stride=1)
        self.conv2 = nn.Conv2d(size_r_5_1[0],size_r_5_1[0],kernel_size=1,stride=1)
        self.conv3 = nn.Conv2d(size_r_4_1[0],size_r_4_1[0],kernel_size=3,stride=3)

    def forward(self, img, img_style):

        f_c_r_4_1,f_c_r_5_1 = self.encoder(img)
        f_s_r_4_1,f_s_r_5_1 = self.encoder(img_style)

        f_cs_r_4_1 = self.conv1(self.SANet_r_4_1(f_c_r_4_1,f_s_r_4_1))
        f_cs_r_5_1 = self.conv2(self.SANet_r_5_1(f_c_r_5_1,f_s_r_5_1))

        f_csc_r_4_1 = f_cs_r_4_1 + f_c_r_4_1
        f_csc_r_5_1 = f_cs_r_5_1 + f_c_r_5_1

        f_csc_r_5_1 = F.interpolate(f_csc_r_5_1.unsqueeze(1),self.size_r_4_1).squeeze(1)

        f_csc_m = f_csc_r_4_1 + f_csc_r_5_1

        f_csc_m = self.conv3(f_csc_m)

        return self.decoder(f_csc_m),f_c_r_4_1,f_c_r_5_1


In [8]:
# INFERENCE

encoder = Encoder(vgg_encoder_r_4_1, vgg_encoder_r_5_1).to(device)
encoder.requires_grad_(False)

style_model = StyleModel(encoder, size_r_4_1, size_r_5_1,decoder).to(device)
style_model.encoder.requires_grad_(False)

style_loss_model = style_loss_model.to(device)

optimimizer = optim.Adam(style_model.parameters(),lr=0.0001)

In [9]:
COCO_PATH = "./test2017/"
STYLE_IMAGE_PATH = "./wave.jpg"
BATCH_SIZE = 1

In [10]:
coco_dataloader = torch.utils.data.DataLoader(LoadCoCoDataset(COCO_PATH, BATCH_SIZE ,IMAGE_SIZE),batch_size=BATCH_SIZE)
style_img = Image.open(STYLE_IMAGE_PATH).convert('RGB')
# center crop image
style_img = ImageOps.fit(style_img,(min(style_img.size),min(style_img.size))).resize(IMAGE_SIZE)
style_img = ToTensor()(style_img).permute(0,2,1).unsqueeze(0).repeat(BATCH_SIZE,1,1,1).to(device)

In [11]:
# INFERENCE

EPOCHS = 10000
ACCUM_GRAD = 2
PRINT_STATS_EVERY = 100
CONTENT_WEIGHT = 25.0
STYLE_WEIGHT = 3.0
IDENTITY_1_WEIGHT = 50.0
IDENTITY_2_WEIGHT = 4000.0
LOSS_STYLE = adaIN
SAVE_PATH = "./SANet_save/"

In [12]:
os.makedirs(SAVE_PATH,exist_ok = True)


content_loss_aggregator = []
style_loss_aggregator = []

for epoch in tqdm(range(EPOCHS)):
    optimimizer.zero_grad()

    for _ in range(ACCUM_GRAD):
        content_img = next(iter(coco_dataloader)).to(device)

        with torch.no_grad():
            _,_,style_features_target = style_loss_model((style_img,[],[]))
        
        prediction, f_c_r_4_1, f_c_r_5_1 = style_model(content_img,style_img)

        f_c_r_4_1, f_c_r_5_1 = normalize_cw(f_c_r_4_1, f_c_r_5_1)
        
        pred_cs_r_4_1,pred_cs_r_5_1 = style_model.encoder(prediction)
        pred_cs_r_4_1, pred_cs_r_5_1 = normalize_cw(pred_cs_r_4_1, pred_cs_r_5_1)

        content_loss = CONTENT_WEIGHT*(torch.norm((pred_cs_r_4_1-f_c_r_4_1).view(BATCH_SIZE,-1), dim=-1).mean() + torch.norm((pred_cs_r_5_1-f_c_r_5_1).view(BATCH_SIZE,-1), dim=-1).mean())

        _,_,style_features= style_loss_model((prediction,[],[]))

        style_loss = 0.0
        for f,f_target,weight in zip(style_features,style_features_target, STYLE_LAYERS_WEIGHTS):
            style_loss += weight*LOSS_STYLE(*normalize(f,f_target)).mean()

        style_loss *= STYLE_WEIGHT

        i_cc,_,_ = style_model(content_img,content_img)
        i_ss,_,_ = style_model(style_img,style_img)

        identity_1_loss = IDENTITY_1_WEIGHT*(torch.norm((i_cc-content_img).view(BATCH_SIZE,-1), dim=-1).mean() + torch.norm((i_ss-style_img).view(BATCH_SIZE,-1), dim=-1).mean())
        _,_,i_cc_style_features= style_loss_model((i_cc,[],[]))
        _,_,content_img_style_features= style_loss_model((content_img,[],[]))
        _,_,i_ss_style_features= style_loss_model((i_ss,[],[]))
        _,_,style_img_style_features= style_loss_model((style_img,[],[]))

        identity_2_loss = 0.0
        for f_icc,f_ic,f_iss,f_is in zip(i_cc_style_features,content_img_style_features,i_ss_style_features,style_img_style_features):
            identity_2_loss += (torch.norm((f_icc-f_ic).view(BATCH_SIZE,-1), dim=-1).mean() + torch.norm((f_iss-f_is).view(BATCH_SIZE,-1), dim=-1).mean())
        identity_2_loss *= IDENTITY_2_WEIGHT

        content_loss_aggregator.append(content_loss.detach().cpu())
        style_loss_aggregator.append(style_loss.detach().cpu())

        loss = content_loss + style_loss + identity_1_loss + identity_2_loss

        loss.backward()

    optimimizer.step()  

    if epoch%PRINT_STATS_EVERY == 0 and not epoch == 0:
        print("Ending epoch: ", str(epoch), " with content loss: ", torch.stack(content_loss_aggregator).mean().numpy().item(),  " and style loss: ", torch.stack(style_loss_aggregator).mean().numpy().item())
        ToPILImage()(content_img[0].permute(0,2,1)).save(SAVE_PATH + "example_input_model.jpg")
        with torch.no_grad():   
            img,_,_ = style_model(content_img[0].unsqueeze(0), style_img[0].unsqueeze(0))
        img = ToPILImage()(img.cpu().squeeze(0).permute(0,2,1))
        img.save(SAVE_PATH + "example_output_model.jpg")
        torch.save(style_model.state_dict(), SAVE_PATH + "model_weights.pth")
        torch.save(optimimizer.state_dict(), SAVE_PATH + "optim_weights.pth")


    
    

  1%|          | 100/10000 [01:02<1:37:40,  1.69it/s]

Ending epoch:  100  with content loss:  79777.1015625  and style loss:  3.8656041622161865


  2%|▏         | 200/10000 [02:00<1:36:03,  1.70it/s]

Ending epoch:  200  with content loss:  78774.6953125  and style loss:  3.3525102138519287


  3%|▎         | 301/10000 [02:59<1:44:57,  1.54it/s]

...

 46%|████▌     | 4600/10000 [45:08<54:23,  1.65it/s]

Ending epoch:  4600  with content loss:  75883.859375  and style loss:  2.024287700653076


 47%|████▋     | 4700/10000 [46:07<53:03,  1.66it/s]  

Ending epoch:  4700  with content loss:  75865.0859375  and style loss:  2.0188217163085938


 47%|████▋     | 4725/10000 [46:22<53:02,  1.66it/s]

In [ ]:
# INFERENCE

style_model.load_state_dict(torch.load(SAVE_PATH + "model_weights.pth",map_location=device))

In [ ]:
# INFERENCE

CONTENT_IMAGE_PATH = "./dragon.jpg"
STYLE_IMAGE_PATH = "./wave.jpg"

In [ ]:
# INFERENCE

content_img = Image.open(CONTENT_IMAGE_PATH).convert('RGB')
# center crop image
content_img = ImageOps.fit(content_img,(min(content_img.size),min(content_img.size))).resize(IMAGE_SIZE)
content_img = ToTensor()(content_img).permute(0,2,1).to(device)

style_img = Image.open(STYLE_IMAGE_PATH).convert('RGB')
# center crop image
style_img = ImageOps.fit(style_img,(min(style_img.size),min(style_img.size))).resize(IMAGE_SIZE)
style_img = ToTensor()(style_img).permute(0,2,1).to(device)

In [ ]:
# INFERENCE

content_style_image,_,_ = style_model(content_img.unsqueeze(0),style_img.unsqueeze(0)).squeeze(0).cpu()
content_style_image = ToPILImage()(content_style_image.permute(0,2,1))
content_style_image.show()